In [1]:
import ee

In [2]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=efi6PgevKkheQOO8D8IKZtwY-pDu-Hn7ddyE4QeZs5s&tc=0q_9ny6OKRzlLclaeyZZIFPPtHdaLvf_tE_nIg-Am78&cc=ejubz13H7m_mfGqvvpUUoHUZD_C_SKEELiDTyTfbA9g

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXkdqeaOftFl0XwfINBLYfhCaTPrjLZ8EUN44woy8kepT7zM0RnT0T4

Successfully saved authorization token.


In [3]:
#Gabon Forest Cover

#Load country data
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
Brazil = countries.filter(ee.Filter.eq('country_na', 'Brazil'))

tree_cover_2022 = ee.Image("UMD/hansen/global_forest_change_2022_v1_10")

In [4]:
loss = tree_cover_2022.select(['loss'])
gain = tree_cover_2022.select(['gain'])

In [5]:
import folium

In [6]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [7]:
# Create a folium map object.
my_map = folium.Map(location=[-14.235, -51.9253], zoom_start = 5)

# Add the layer to the map object.
my_map.add_ee_layer(gain.clip(Brazil).updateMask(gain),{'palette': ['blue']},'Brazil Forest Gain')
my_map.add_ee_layer(loss.clip(Brazil).updateMask(loss),{'palette': ['red']},'Brazil Forest Loss')

my_map.add_ee_layer(ee.Image().paint(Brazil, 0, 1), {'palette': ['grey']}, 'Brazil')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)